In [1]:
from huggingface_hub import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# Choose your model (e.g., TinyLlama)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
local_dir = "./model"  # Where to save

# Download and save
snapshot_download(repo_id=model_name, local_dir=local_dir)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

eval_results.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

'/home/cdrleo/Desktop/ai_engineering/01_deep_learning_applications/00_notes/random/simple_agent/model'

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

2025-01-27 21:05:38.427784: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 21:05:38.992082: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-27 21:05:40.648844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/cdrleo/.local/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change acc

In [2]:
local_model_path = "./model"  # Path to your saved model

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    device_map="cpu",
    torch_dtype=torch.float32
)

In [3]:
# Create optimized pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device="cpu",  # Explicit CPU (ValueError: The model has been loaded with `accelerate` and therefore cannot be moved to a specific device. Please discard the `device` argument when creating your pipeline object)
    torch_dtype=torch.float32,
)

In [4]:
def generate_response(prompt):
    formatted_prompt = f"""<|system|>
    You are a coding assistant for Python, Linux, and ML. Give short answers.</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    """
    output = pipe(
        formatted_prompt,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    return output[0]["generated_text"].split("<|assistant|>")[-1].strip()


In [5]:
print("Ask about coding (type 'exit'):")
while True:
    user_input = input("> ")
    if user_input.lower() == "exit":
        break
    print(f"Question: {user_input}")
    print(f"Answer:\n{generate_response(user_input)}")
    print(10*"-")
    

Ask about coding (type 'exit'):
Question: What is a linear regression?
Answer:
Linear regression is a statistical model that estimates the relationship between a dependent variable (e.g. Y) and a set of independent variables (x). The dependent variable represents the outcome of an event or the outcome of a business or product. The independent variables are the factors that affect the outcome and include things like age, gender, income, education, or other factors. The model is represented by a line that best predicts the y value(s) given the x values. Linear regression can be used to estimate the relationship between a set of variables and to find the values of the independent variables that willющеve the best fit.
----------
